<a href="https://colab.research.google.com/github/AeroEng16/FootballAnalysis/blob/main/lineDetectionAndHomography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Found existing installation: opencv-contrib-python 4.5.5.62
Uninstalling opencv-contrib-python-4.5.5.62:
  Successfully uninstalled opencv-contrib-python-4.5.5.62
  Using cached opencv_contrib_python-4.5.5.62-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
Using cached opencv_contrib_python-4.5.5.62-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (66.6 MB)


In [ ]:
cv2.videoCapture("https://drive.google.com/file/d/1fQJJtbJNqtRfn6FjnXryUw0r-icxHwNs/view?usp=drive_link")

'4.10.0'